In [ ]:
import numpy as np
import pandas as pd
import pickle
import json

from sklearn.metrics.cluster import adjusted_mutual_info_score
from gensim.models import LdaSeqModel

import DDMM class by reading

In [ ]:
### define DDMM model class

class DDMM():
    
    #Miscellaneous function to speed up computation (i.e. by avoiding for loops)
    def multirange(self, counts):
        counts = np.asarray(counts)
        # Remove the following line if counts is always strictly positive.
        counts = counts[counts != 0]
        counts1 = counts[:-1]
        reset_index = np.cumsum(counts1)

        incr = np.ones(counts.sum(), dtype=int)
        incr[0] = 0
        incr[reset_index] = 1 - counts1

        # Reuse the incr array for the final result.
        incr.cumsum(out=incr)
        return np.tile(incr, (self.K,1)).T        
    
    # DDMM implementation
    def __init__(self, K, T, vocab, bows, alpha = None, beta = None):
        
        self.K = K
        self.vocab = vocab
        self.T = T
        self.bows = bows
        if alpha == None:
            self.alpha = 1.3
        else:
            self.alpha = alpha
        if beta == None:
            self.beta = 0.02
        else:
            self.beta = beta
        
        # create cluster assignment for each time step
        self.clust_asgn = []
        
        # create number of epochs to convergence for each time step
        self.epoch_conv = []
        
        # create epoch similarity for each time step
        self.epoch_similarity = []
        
        # create alphas
        self.alphas = []
        
        # create betas
        self.betas = []
        
        # create number of large enough clusters
        self.n_large_clusts = []
        
    def fit(self, max_epochs = 5, conv_crit = 0.97, lbda = 1, mu = 1):
        
        #initialize vocab length and hyperparameters
        V = len(self.vocab)
        alpha_ = self.alpha * np.ones(self.K)
        beta_ = self.beta * np.ones((self.K, V))
        
        #iterate through each timestep
        for t in range(self.T):
            
            n_large_clust = [self.K]
            
            alpha_copy = copy.deepcopy(alpha_)
            beta_copy = copy.deepcopy(beta_)
            self.alphas.append(alpha_copy) #update alphas
            self.betas.append(beta_copy) #update beta
            
            bow = self.bows[t] #get bow for time t
            bow_np = np.array(bow) #convert to numpy array
            D = bow.shape[0] #get document size
            clusters = np.random.choice(self.K, D) #cluster initialization
            
            #compute mk, nk and nkv for each k,v
            mk = np.bincount(clusters, minlength = self.K)
            nk = np.bincount(clusters, weights = np.sum(bow, axis = 1), minlength = self.K).astype(int)
            nkv = np.apply_along_axis(lambda x: np.bincount(clusters, weights = x, 
                                                            minlength = self.K), axis = 0, arr = bow)
            
            #number of words per document
            Nd = np.sum(bow, axis = 1)
            
            #initialize number of epochs and within-epoch similarity (to compare with conv_crit)
            n_epoch = 0
            epoch_sim = 0
            
            #repeat until convergence or maximum epochs
            while n_epoch < max_epochs and epoch_sim < conv_crit:
                print('Starting epoch', n_epoch + 1, 'for time step', t)
                curr_clust = copy.deepcopy(clusters) #take note of current cluster
                
                for d in range(D):
                    
                    #knock out d from its cluster; let say z is d's current cluster
                    z = clusters[d] #get d's current cluster
                    #modify mk, nk and nkv
                    mk[z] -= 1
                    nk[z] -= Nd[d]
                    nkv[z] -= bow_np[d]
                    
                    #sample a new cluster for d from Equation 15 (use log form)
                    #ignore sum(alpha_k) + D - 1 since it does not depend on k

                    #first component: log of alpha_k + m_k
                    first = np.log(alpha_ + mk)

                    #second component: sum from v=1 to V; j = 1 to Ndv of log(beta_kv + n_kv + j - 1)
                    temp = (beta_ + nkv).T
                    rep = bow_np[d]
                    temp1 = np.vstack(temp)
                    temp1 = temp1[np.repeat(np.arange(temp1.shape[0]), rep)]
                    temp2 = self.multirange(rep)
                    second = np.sum(np.log(temp1 + temp2), axis = 0)

                    #third component: sum from i=1 to Nd of log(sum of beta_kv for each v + n_k + i - 1)
                    Ndd = Nd[d]
                    temp = np.sum(beta_, axis = 1) + nk
                    third = np.sum(np.log(np.tile(temp, (Ndd, 1)) + \
                                          np.tile(np.arange(0, Ndd, 1), (self.K, 1)).T), axis = 0)

                    #sample a cluster from the proportions; let the cluster be q
                    prop = softmax(first + second - third)
                    q = np.random.choice(np.arange(self.K), p = prop) #new cluster assignment for d

                    #update mq, nq and nqv
                    mk[q] += 1
                    nk[q] += Nd[d]
                    nkv[q] += bow_np[d]

                    #update cluster assignment
                    clusters[d] = q
                
                n_epoch += 1 #update number of epoch
                epoch_sim = np.sum(curr_clust == clusters)/D #update cluster similarity for this epoch
                
                clust_prop = np.bincount(clusters, minlength = self.K)/D
                n_large_clust.append(len(clust_prop[clust_prop > 0.01]))
                
            #update tracker variables
            self.clust_asgn.append(clusters)
            self.epoch_conv.append(n_epoch)
            self.epoch_similarity.append(epoch_sim)
            
            #update alpha and beta
            
            #compute mk, nk and nkv for each k,v
            mk = np.bincount(clusters, minlength = self.K)
            nk = np.bincount(clusters, weights = np.sum(bow, axis = 1), minlength = self.K).astype(int)
            nkv = np.apply_along_axis(lambda x: np.bincount(clusters, weights = x, 
                                                            minlength = self.K), axis = 0, arr = bow)
            
            #number of words per document
            Nd = np.sum(bow, axis = 1)
            
            #modify alpha and beta
            alpha_ += lbda * mk
            beta_ += mu * nkv
            
            #update number of large clusters
            self.n_large_clusts.append(n_large_clust)

    def cluster_size(self):
        return [np.bincount(x, minlength = self.K) for x in self.clust_asgn]
    
    def top_words(self, n = 10):
        result = {}
        for t in range(self.T):
            bow = self.bows[t]
            clusters = self.clust_asgn[t]
            result_t = []
            for k in range(self.K):
                result_t.append(np.array(self.vocab)[np.array(bow.iloc[np.where(clusters == k)].sum(axis = 0) \
                                                    .sort_values()[::-1].index[:n]).astype(int)])
            result[t] = result_t
        return result

Check ODDMM Topic Evolution

In [ ]:
# Read trained ODDMM
with open('title_tags_description_oddmm_k16_run1', 'rb') as model:
    ddmm = pickle.load(model)

# Check top words
ddmm_top_words = ddmm.top_words()

In [ ]:
# Collect topic words for cluster k
for k in range(16):
    pd.DataFrame([ddmm_top_words[i][k] for i in range(0,8)]).T.to_csv('title_tags_description_oddmm_cluster' + str(k) + '_top10words.csv', index=False)

Check DTM Topic Evolution

In [ ]:
# Read DTM
dtm = LdaSeqModel.load('title_tags_description_ldaseq_k16_run1')

In [ ]:
# Read index to word json file
### Warning: the keys (word indices) are string type
with open('title_tags_description/title_tags_description_indexword.json') as json_file:
    indexword_dict = json.load(json_file)

In [ ]:
# Collect topic words for cluster k
for k in range(16):
    timesteps = 8
    top_count = 10
    topic_words = dtm.print_topic_times(k, top_terms=top_count)
    across_timesteps = []
    for i in range(timesteps):
        within_timesteps = []
        for j in range(top_count):
            within_timesteps.append(indexword_dict[topic_words[i][j][0]])
        across_timesteps.append(within_timesteps)
    #pd.DataFrame(across_timesteps).T.to_csv('title_tags_description_dtm_cluster' + str(k) + '_top10words.csv', index=False)

In [ ]:
pd.DataFrame(across_timesteps).T

Calculate AMI for DDMM

In [ ]:
# Load trained model
with open('title_tags_description' + '/' + 'title_tags_description' + '_ddmm_' + str(16) + '.pkl', 'rb') as model:
    ddmm = pickle.load(model)
    
# Read meta file
meta_genres = pd.read_csv('title_tags_description/title_tags_description_meta.csv', \
                   lineterminator='^', dtype={'genre':'category'}).genre.cat.codes

calculate_ami(ddmm.clust_asgn, meta_genres)

Calculate AMI for DTM

In [ ]:
# Load trained model
ldaseq = LdaSeqModel.load('title_ldaseq_k16')

# Read BoW file
bow = np.load('title/title_bow.npy')

# Read index to word json file
### Warning: the keys (word indices) are string type
with open('title/title_indexword.json') as json_file:
    indexword_dict = json.load(json_file)
    
# Read meta file
meta_genres = pd.read_csv('title/title_meta.csv', \
                   lineterminator='^', dtype={'genre':'category'}).genre.cat.codes

# Create corpus format from bow for ldaseq
corpus = []
for i in range(len(bow)):
    doc_corpus = []
    for ind in list(np.where(bow[i]>0)[0]):
        doc_corpus.append((ind, bow[i][ind]))
    corpus.append(doc_corpus)
    
# Function to get ldaseq assigned clusters for documents
def get_ldaseq_clusters(ldaseq, corpus):
    clusters_list = []
    for i in range(len(corpus)):
        clusters_list.append(np.argmax(ldaseq.doc_topics(i)))
    return clusters_list

# Get ldaseq assigned clusters
clusters = get_ldaseq_clusters(ldaseq, corpus)

# Hack to make the AMI function work for ldaseq (since the AMI function requires multi-dim array)
clusters = np.array(clusters).reshape(2, -1)

def calculate_ami(clust_asgn, true_genres):
    pred_clusters = np.concatenate(clust_asgn).ravel()
    return adjusted_mutual_info_score(true_genres, pred_clusters)

calculate_ami(clusters, meta_genres)

Graph for cluster convergence/similarity

In [ ]:
versions_list = ['title',
                 'title_tags',
                 'title_tags_description']
markers = ['o', 'v', '^']
K = 16

plt.figure()
    
for i, version_name in enumerate(versions_list):

    with open(version_name + '_oddmm_' + 'k16_run1.pkl', 'rb') as model:
        ddmm = pickle.load(model)

    temp = ddmm.epoch_similarity
    plt.plot(np.arange(len(temp)), temp, marker = markers[i], label = version_name);

plt.legend(loc=(1.04,0.4));
plt.yticks(np.arange(0.84,1,0.02));
plt.xlabel('Time step (t)');
plt.ylabel('Cluster similarity (4th and 5th epochs)');
plt.figtext(0.95, 0.3, 'Algorithm is run for 5 epochs');
plt.title('Cluster similarity for each time step (K = ' + str(K) + ')');

Graph for AMI

In [ ]:
versions_list = ['title', 'tags', 'description', 
                 'title_tags', 'title_description', 
                 'tags_description', 'title_tags_description']
markers = ['o', 'v', '^', '<', '>', 's', 'X']
K_list = [8,16,32]

version_ami_lists = []
for version_name in version_list:
    temp = []
    meta_genres = pd.read_csv(version_name + '/' + version_name + '_meta.csv', lineterminator='^', \
                      dtype={'genre':'category'}).genre.cat.codes
    for K in K_list:
        temp2 = []
        for run in range(1,6):
            with open(version_name + '_oddmm_' + 'k' + str(K) + '_run' + str(run) + '.pkl', 'rb') as model:
                ddmm = pickle.load(model)
            temp2.append(calculate_ami(ddmm.clust_asgn, meta_genres))
        temp.append(np.mean(temp2))
    version_ami_lists.append(temp)

for i, version_name in enumerate(versions_list):
    version_ami_list = version_ami_lists[i]
    plt.plot([0,1,2], version_ami_list, label = version_name, marker = markers[i]);
        
plt.xticks([0,1,2], K_list)
plt.legend(loc=(1.04, 0.48));
plt.xlabel('Number of clusters (K)');
plt.ylabel('Adjusted mutual information (AMI)');
plt.figtext(0.95, 0.38, 'AMI scores are averaged over 5 runs');
plt.title('Adjusted mutual information (AMI) with different number of clusters');